In [23]:
import requests
from bs4 import BeautifulSoup
import string

In [13]:
movie_urls = []

In [24]:
alphabet = list(string.ascii_lowercase)

In [19]:
base_url = 'http://www.boxofficemojo.com/movies/alphabetical.htm?'

In [20]:
def create_url(letter,page):
    '''this function takes in the query params letter and page 
       and returns the full url string
    '''
    q_letter = 'letter=%s' % letter
    q_page = 'page=%s' % page
    url = base_url + q_letter + '&' + q_page
    return url

In [26]:
for letter in alphabet[:5]:
    page = 1
    while page < 5:
        print create_url(letter,page)
        page += 1

http://www.boxofficemojo.com/movies/alphabetical.htm?letter=a&page=1
http://www.boxofficemojo.com/movies/alphabetical.htm?letter=a&page=2
http://www.boxofficemojo.com/movies/alphabetical.htm?letter=a&page=3
http://www.boxofficemojo.com/movies/alphabetical.htm?letter=a&page=4
http://www.boxofficemojo.com/movies/alphabetical.htm?letter=b&page=1
http://www.boxofficemojo.com/movies/alphabetical.htm?letter=b&page=2
http://www.boxofficemojo.com/movies/alphabetical.htm?letter=b&page=3
http://www.boxofficemojo.com/movies/alphabetical.htm?letter=b&page=4
http://www.boxofficemojo.com/movies/alphabetical.htm?letter=c&page=1
http://www.boxofficemojo.com/movies/alphabetical.htm?letter=c&page=2
http://www.boxofficemojo.com/movies/alphabetical.htm?letter=c&page=3
http://www.boxofficemojo.com/movies/alphabetical.htm?letter=c&page=4
http://www.boxofficemojo.com/movies/alphabetical.htm?letter=d&page=1
http://www.boxofficemojo.com/movies/alphabetical.htm?letter=d&page=2
http://www.boxofficemojo.com/movie